In [98]:
import pandas as pd
import numpy as np
import re

In [99]:
checked_domains_file = 'CheckedCompanies_20191206.csv'
emails_file = 'DistinctEmails_20191125.csv'

In [100]:
emails = pd.read_csv(emails_file)
checked = pd.read_csv(checked_domains_file)

In [101]:
emails['Status'] = ''

In [103]:
emails = emails[pd.notnull(emails['Domain'])]

In [104]:
emails.reset_index(inplace=True)

In [106]:
emails = emails.iloc[:,1:]

In [108]:
emails_array = emails.to_numpy()

In [109]:
for i in range(emails_array.shape[0]):
    emails_array[i][0] = '@{}'.format(emails_array[i][0])

In [110]:
after_underscore = checked_domains_file.find('_')+1
before_ext = checked_domains_file.find('.csv')
checked_file_date = checked_domains_file[after_underscore:before_ext]
checked_domains = pd.read_csv(checked_domains_file)
cols = ['Date','Company Name(s)','Email Domain','Industry Segment','Respective Sub-Industry Segment']
checked_domains = checked_domains[cols]
# Splitting company name(s) attribute and email domains
for i in range(checked_domains.shape[0]):
    co_names = str(checked_domains.iloc[i][1]).lower().split(';')
    e_domains = str(checked_domains.iloc[i][2]).lower().split(',')
    checked_domains.iloc[i][1] = co_names
    checked_domains.iloc[i][2] = e_domains
checked_domains['Date'] = pd.to_datetime(checked_domains['Date'],errors='coerce').dt.date
checked_domains.sort_values('Date',ascending=False,inplace=True)
checked_domains.reset_index(drop=True,inplace=True)
checked_domains.dropna(subset=['Date'],inplace=True)
checked_array = checked_domains.to_numpy()
for i in range(checked_array.shape[0]):
    checked_array[i][0] = checked_array[i][0].strftime('%Y%m%d')
    
    domain_list = checked_array[i][2]
    for d in range(len(domain_list)):
        domain_list[d] = domain_list[d].strip()
    checked_array[i][2] = domain_list
    
    company_list = checked_array[i][1]
    for c in range(len(company_list)):
        #company_list[c] = company_list[c].translate(None, string.punctuation)
        company_list[c] = re.sub(r"[(,.;@#?!&$)]+\ *"," ", company_list[c])
        company_list[c] = ' '.join(company_list[c].split())
        #company_list[c] = cleanco(company_list[c].strip()).clean_name()
    checked_array[i][1] = company_list

In [111]:
checked_array[1]

array(['20191205', list(['pitney bowes']), list(['@pb.com']),
       'Technology', nan], dtype=object)

In [112]:
cel = []

In [113]:
for i in range(checked_array.shape[0]):
    for j in range(len(checked_array[i][2])):
        cel.append(checked_array[i][2][j])

In [115]:
cel = set(cel)

In [119]:
for i in range(emails_array.shape[0]):
    if emails_array[i][0] in cel:
        emails_array[i][2] = 'Checked'
    else:
        emails_array[i][2] = ''

In [120]:
emails_array[4]

array(['@ryder.com', 1022, 'Checked'], dtype=object)

In [121]:
filled_array = pd.DataFrame(data=emails_array)

In [123]:
filled_array.to_csv('DomainStatusUpdated_{}.csv'.format(checked_file_date))